<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Goal?" data-toc-modified-id="Goal?-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Goal?</a></span></li><li><span><a href="#Import-modules" data-toc-modified-id="Import-modules-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Import modules</a></span></li><li><span><a href="#Load-the-dataset" data-toc-modified-id="Load-the-dataset-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load the dataset</a></span></li><li><span><a href="#Split-the-data" data-toc-modified-id="Split-the-data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Split the data</a></span></li><li><span><a href="#Forecast-evaluation" data-toc-modified-id="Forecast-evaluation-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Forecast evaluation</a></span></li><li><span><a href="#Data-cleaning" data-toc-modified-id="Data-cleaning-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Data cleaning</a></span></li><li><span><a href="#Input-output-preparations" data-toc-modified-id="Input-output-preparations-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Input-output preparations</a></span></li><li><span><a href="#References" data-toc-modified-id="References-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>References</a></span></li></ul></div>

# Introduction

<div class="alert alert-block alert-warning">
<font color=black><br>

**What?** Multivariate Multi-Step Time Series Forecasting Models for Air Pollution

<br></font>
</div>

# Goal?

<div class="alert alert-block alert-info">
<font color=black><br>

- The **Air Quality Prediction** dataset describes weather conditions at multiple sites.
- The goal is to predict the air quality measurements over the subsequent three days.
- This is a multi-step univariate time series analysis, because we have only one value recorder per time step but this depends on many inputs.
- A **naive forecasts model** is develop which can be used as a baseline model. This is generally done to establish if the model is approprioate for this problem or note.

<br></font>
</div>

# Import modules

In [31]:
from numpy import unique
from numpy import nan
from numpy import array
from numpy import savetxt
from pandas import read_csv
import pandas as pd

# Load the dataset

<div class="alert alert-block alert-info">
<font color=black><br>

- The Air Quality Prediction dataset describes weather conditions at multiple sites and requires a prediction of air quality measurements over the subsequent three days.

- Specifically, weather observations such as temperature, pressure, wind speed, and wind direction are provided hourly for eight days for multiple sites. The objective is to predict air quality measurements for the next 3 days at multiple sites. The forecast lead times are not contiguous; instead, specific lead times must be forecast over the 72 hour forecast period. They are:
`+1, +2, +3, +4, +5, +10, +17, +24, +48, +72`

- Further, the dataset is divided into disjoint but contiguous chunks of data, with eight days of data followed by three days that require a forecast.

- Not all observations are available at all sites or chunks and not all output variables are available at all sites and chunks. There are large portions of missing data that must be addressed.

<br></font>
</div>

In [3]:
dataset = read_csv('../DATASETS/air_quality_dataset/TrainingData.csv', header = 0)

In [17]:
dataset.head(10)

,rowID,chunkID,position_within_chunk,month_most_common,weekday,hour,Solar.radiation_64,WindDirection..Resultant_1,WindDirection..Resultant_1018,WindSpeed..Resultant_1,...,target_4_6006,target_4_8003,target_5_6006,target_7_57,target_8_57,target_8_4002,target_8_6004,target_8_8003,target_9_4002,target_9_8003
0,1,1,1,10,Saturday,21,0.01,117.0,187.0,0.3,...,1.748424,NaN,NaN,5.130631,1.341606,2.138792,3.013752,NaN,5.679280,NaN
1,2,1,2,10,Saturday,22,0.01,231.0,202.0,0.5,...,2.144120,NaN,NaN,5.130631,1.195779,2.722099,3.888712,NaN,7.426751,NaN
2,3,1,3,10,Saturday,23,0.01,247.0,227.0,0.5,...,1.932469,NaN,NaN,5.136395,1.409658,3.110970,3.888712,NaN,7.683732,NaN
3,4,1,4,10,Sunday,0,0.01,219.0,218.0,0.2,...,2.088907,NaN,NaN,5.217102,1.477711,2.041574,3.208188,NaN,4.831243,NaN
4,5,1,5,10,Sunday,1,0.01,2.0,216.0,0.2,...,2.604232,NaN,NaN,5.217102,1.458267,2.138792,3.499841,NaN,4.625658,NaN
5,6,1,6,10,Sunday,2,0.01,288.0,2.0,0.3,...,2.687052,NaN,NaN,5.170984,1.604094,2.236010,3.305406,NaN,5.833469,NaN
6,7,1,7,10,Sunday,3,0.01,330.0,8.0,0.3,...,2.677850,NaN,NaN,2.951554,1.555485,1.652703,2.430445,NaN,3.854715,NaN
7,8,1,8,10,Sunday,4,0.01,316.0,4.0,0.8,...,2.549019,NaN,NaN,2.951554,1.711033,1.458267,1.361049,NaN,2.595508,NaN
8,9,1,9,10,Sunday,5,0.01,285.0,342.0,0.7,...,2.226941,NaN,NaN,2.928495,1.623537,1.361049,1.263832,NaN,2.621206,NaN
9,10,1,10,10,Sunday,6,0.05,337.0,352.0,1.3,...,1.996885,NaN,NaN,1.815897,1.380493,1.263832,1.166614,NaN,2.004452,NaN


In [7]:
# We can see there are quite a lot of missing values
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37821 entries, 0 to 37820
Data columns (total 95 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   rowID                          37821 non-null  int64  
 1   chunkID                        37821 non-null  int64  
 2   position_within_chunk          37821 non-null  int64  
 3   month_most_common              37821 non-null  int64  
 4   weekday                        37821 non-null  object 
 5   hour                           37821 non-null  int64  
 6   Solar.radiation_64             37395 non-null  float64
 7   WindDirection..Resultant_1     36391 non-null  float64
 8   WindDirection..Resultant_1018  9756 non-null   float64
 9   WindSpeed..Resultant_1         36391 non-null  float64
 10  WindSpeed..Resultant_1018      9756 non-null   float64
 11  Ambient.Max.Temperature_14     3219 non-null   float64
 12  Ambient.Max.Temperature_22     7224 non-null  

In [53]:
# How many days are there in each chunk?
for i in range(211):
    print(i, " how many days?", len(dataset[dataset["chunkID"] == i]["weekday"].unique()))

0  how many days? 0
1  how many days? 7
2  how many days? 6
3  how many days? 7
4  how many days? 6
5  how many days? 7
6  how many days? 7
7  how many days? 7
8  how many days? 7
9  how many days? 7
10  how many days? 7
11  how many days? 7
12  how many days? 7
13  how many days? 7
14  how many days? 6
15  how many days? 7
16  how many days? 4
17  how many days? 7
18  how many days? 7
19  how many days? 5
20  how many days? 7
21  how many days? 7
22  how many days? 7
23  how many days? 7
24  how many days? 6
25  how many days? 7
26  how many days? 7
27  how many days? 7
28  how many days? 7
29  how many days? 6
30  how many days? 7
31  how many days? 6
32  how many days? 6
33  how many days? 7
34  how many days? 7
35  how many days? 7
36  how many days? 6
37  how many days? 7
38  how many days? 7
39  how many days? 7
40  how many days? 7
41  how many days? 7
42  how many days? 7
43  how many days? 7
44  how many days? 7
45  how many days? 7
46  how many days? 7
47  how many days? 7
48

In [61]:
# How many days are there in each chunk? 192 (8 days * 24 hours), that is why we say there are 8 days.
for i in range(211):
    print(i, " how many days?", len(dataset[dataset["chunkID"] == i]["position_within_chunk"].unique()))

0  how many days? 0
1  how many days? 192
2  how many days? 168
3  how many days? 192
4  how many days? 144
5  how many days? 192
6  how many days? 192
7  how many days? 192
8  how many days? 192
9  how many days? 192
10  how many days? 192
11  how many days? 192
12  how many days? 192
13  how many days? 192
14  how many days? 168
15  how many days? 192
16  how many days? 89
17  how many days? 192
18  how many days? 192
19  how many days? 120
20  how many days? 168
21  how many days? 168
22  how many days? 192
23  how many days? 192
24  how many days? 134
25  how many days? 192
26  how many days? 192
27  how many days? 192
28  how many days? 192
29  how many days? 165
30  how many days? 192
31  how many days? 168
32  how many days? 168
33  how many days? 192
34  how many days? 192
35  how many days? 192
36  how many days? 168
37  how many days? 192
38  how many days? 192
39  how many days? 192
40  how many days? 175
41  how many days? 192
42  how many days? 192
43  how many days? 192
4

<div class="alert alert-block alert-info">
<font color=black><br>

- When working with the **naive models**, we are only interested in the target variables, and **none** of the input meteorological variables. 

<br></font>
</div>

In [66]:
# How many target variables do we have?
dataset.filter(regex = 'target_*')

,target_1_57,target_10_4002,target_10_8003,target_11_1,target_11_32,target_11_50,target_11_64,target_11_1003,target_11_1601,target_11_4002,...,target_4_6006,target_4_8003,target_5_6006,target_7_57,target_8_57,target_8_4002,target_8_6004,target_8_8003,target_9_4002,target_9_8003
0,2.679233,6.181623,NaN,0.114975,0.114975,0.114975,0.114975,0.114975,0.114975,0.114975,...,1.748424,NaN,NaN,5.130631,1.341606,2.138792,3.013752,NaN,5.679280,NaN
1,2.679233,8.475833,NaN,0.114975,0.114975,0.114975,0.114975,0.114975,0.114975,0.114975,...,2.144120,NaN,NaN,5.130631,1.195779,2.722099,3.888712,NaN,7.426751,NaN
2,2.679233,8.921930,NaN,0.114975,0.114975,0.114975,0.114975,0.114975,0.114975,0.114975,...,1.932469,NaN,NaN,5.136395,1.409658,3.110970,3.888712,NaN,7.683732,NaN
3,2.679233,5.098246,NaN,0.114975,0.114975,0.114975,0.114975,0.114975,0.114975,0.114975,...,2.088907,NaN,NaN,5.217102,1.477711,2.041574,3.208188,NaN,4.831243,NaN
4,2.679233,4.875197,NaN,0.114975,0.114975,0.114975,0.114975,0.114975,0.114975,0.114975,...,2.604232,NaN,NaN,5.217102,1.458267,2.138792,3.499841,NaN,4.625658,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37816,NaN,0.095592,NaN,0.919801,1.782115,NaN,1.609652,1.322214,0.114975,0.574876,...,1.453953,NaN,0.933143,NaN,NaN,0.291653,0.291653,NaN,0.642453,NaN
37817,NaN,0.350504,NaN,0.804826,1.724628,NaN,1.379702,0.747339,0.114975,0.114975,...,1.076661,NaN,0.599878,NaN,NaN,0.291653,0.291653,NaN,1.002226,NaN
37818,NaN,0.254912,NaN,0.689851,1.609652,NaN,NaN,0.632363,0.114975,0.287438,...,1.113470,NaN,0.599878,NaN,NaN,0.291653,0.291653,NaN,0.822339,NaN
37819,NaN,0.286776,NaN,0.459901,NaN,NaN,1.092264,0.919801,0.114975,0.459901,...,1.058257,NaN,0.666531,NaN,NaN,0.291653,0.291653,NaN,0.719547,NaN


In [11]:
# group data by chunks
values = dataset.values

In [14]:
print(dataset.shape)
print(values.shape)

(37821, 95)
(37821, 95)


In [15]:
#chunk_ids = dataset["chunkID"].unique()
chunk_ids = unique(values[:, 1])
chunk_ids

array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
       71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87,
       88, 89, 90, 91, 92, 93, 95, 96, 97, 98, 99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182,
       183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195,
       196, 197, 198, 199, 200, 201, 202, 203, 204,

<div class="alert alert-block alert-info">
<font color=black><br>

- We can group data by the ‘chunkID’ variable.
- To do so a function is created.

<br></font>
</div>

In [18]:
# split the dataset by 'chunkID', return a dict of id to rows
def to_chunks(values, chunk_ix=1):
    """Split the dataset into chunks.
    
    Returns a dict of ID to rows
    """
    chunks = dict()
    # get the unique chunk ids
    chunk_ids = unique(values[:, chunk_ix])
    # group rows by chunk id
    for chunk_id in chunk_ids:
        selection = values[:, chunk_ix] == chunk_id
        chunks[chunk_id] = values[selection, :]
    return chunks

In [19]:
chunks = to_chunks(values)
print('Total Chunks: %d' % len(chunks))

Total Chunks: 208


In [32]:
chunks.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210])

In [33]:
# Inspecting the 100th chunk
pd.DataFrame(chunks[100])

,0,1,2,3,4,5,6,7,8,9,...,85,86,87,88,89,90,91,92,93,94
0,26137,100,1,1,Thursday,2,0.01,278.0,NaN,4.1,...,2.107311,NaN,NaN,NaN,NaN,0.291653,0.486089,NaN,1.07932,NaN
1,26138,100,2,1,Thursday,3,0.01,293.0,NaN,3.7,...,2.410985,NaN,NaN,NaN,NaN,0.291653,0.486089,NaN,0.848037,NaN
2,26139,100,3,1,Thursday,4,0.01,278.0,NaN,2.9,...,2.613434,NaN,NaN,NaN,NaN,0.291653,0.583307,NaN,0.925132,NaN
3,26140,100,4,1,Thursday,5,0.01,252.0,NaN,4.0,...,2.226941,NaN,NaN,NaN,NaN,0.291653,0.680525,NaN,0.796641,NaN
4,26141,100,5,1,Thursday,6,0.01,260.0,NaN,4.3,...,2.199334,NaN,NaN,NaN,NaN,0.291653,0.972178,NaN,1.053622,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,26324,100,188,1,Thursday,21,0.01,290.0,NaN,4.4,...,4.205421,NaN,NaN,NaN,NaN,0.291653,0.291653,NaN,0.719547,NaN
188,26325,100,189,1,Thursday,22,0.01,314.0,NaN,6.4,...,3.570467,NaN,NaN,NaN,NaN,0.291653,0.291653,NaN,0.41117,NaN
189,26326,100,190,1,Thursday,23,0.01,320.0,NaN,5.6,...,3.248388,NaN,NaN,NaN,NaN,0.291653,0.291653,NaN,0.359773,NaN
190,26327,100,191,1,Friday,0,0.01,322.0,NaN,5.1,...,2.944715,NaN,NaN,NaN,NaN,0.291653,0.291653,NaN,0.436868,NaN


In [60]:
# I still see a cycle of 7 days not 8!
unique(pd.DataFrame(chunks[100][:,4]))

array(['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday',
       'Wednesday'], dtype=object)

# Split the data

<div class="alert alert-block alert-info">
<font color=black><br>

- Each chunk covers an interval of **8** days of hourly observations, although the number of actual observations within each chunk may vary widely.
- We can split each chunk into the first **5** days of observations for training and the last three for test.
- Each observation has a row called ‘position_within_chunk‘ that varies from 1 to 192 (**8 days * 24 hours**). We can therefore take all rows with a value in this column that is less than or equal to 120 (5 * 24) as training data and any values more than 120 as test data.
- Further, any chunks that don’t have any observations in the train or test split can be dropped as not viable.
- We do not require the entire test dataset; instead, we only require the observations at specific lead times over the three day period, specifically the lead times: `+1, +2, +3, +4, +5, +10, +17, +24, +48, +72`

- When working with the **naive models**, we are only interested in the target variables, and **none** of the input meteorological variables. 
- Therefore, we can remove the input data and have the train and test data **only** comprised of the 39 target variables for each chunk, as well as the position within chunk and hour of observation.

<br></font>
</div>

In [68]:
# split each chunk into train/test sets
def split_train_test(chunks, row_in_chunk_ix=2):
    train, test = list(), list()
    # first 5 days of hourly observations for train
    cut_point = 5 * 24
    # enumerate chunks
    for k,rows in chunks.items():
        # split chunk rows by 'position_within_chunk'
        train_rows = rows[rows[:,row_in_chunk_ix] <= cut_point, :]
        test_rows = rows[rows[:,row_in_chunk_ix] > cut_point, :]
        if len(train_rows) == 0 or len(test_rows) == 0:
            print('>dropping chunk=%d: train=%s, test=%s' % (k, train_rows.shape, test_rows.shape))
            continue
        # store with chunk id, position in chunk, hour and all targets
        indices = [1,2,5] + [x for x in range(56,train_rows.shape[1])]
        train.append(train_rows[:, indices])
        test.append(test_rows[:, indices])
    return train, test

In [69]:
# return a list of relative forecast lead times
def get_lead_times():
    return [1, 2 ,3, 4, 5, 10, 17, 24, 48, 72]

In [70]:
# convert the rows in a test chunk to forecasts
def to_forecasts(test_chunks, row_in_chunk_ix=1):
    # get lead times
    lead_times = get_lead_times()
    # first 5 days of hourly observations for train
    cut_point = 5 * 24
    forecasts = list()
    # enumerate each chunk
    for rows in test_chunks:
        chunk_id = rows[0, 0]
        # enumerate each lead time
        for tau in lead_times:
            # determine the row in chunk we want for the lead time
            offset = cut_point + tau
            # retrieve data for the lead time using row number in chunk
            row_for_tau = rows[rows[:,row_in_chunk_ix]==offset, :]
            # check if we have data
            if len(row_for_tau) == 0:
                # create a mock row [chunk, position, hour] + [nan...]
                row = [chunk_id, offset, nan] + [nan for _ in range(39)]
                forecasts.append(row)
            else:
                # store the forecast row
                forecasts.append(row_for_tau[0])
    return array(forecasts)

In [71]:
# split into train/test
train, test = split_train_test(chunks)

# flatten training chunks to rows
train_rows = array([row for rows in train for row in rows])

# print(train_rows.shape)
print('Train Rows: %s' % str(train_rows.shape))

# reduce train to forecast lead times only
test_rows = to_forecasts(test)
print('Test Rows: %s' % str(test_rows.shape))

# save datasets
#savetxt('AirQualityPrediction/naive_train.csv', train_rows, delimiter=',')
#savetxt('AirQualityPrediction/naive_test.csv', test_rows, delimiter=',')

>dropping chunk=69: train=(0, 95), test=(28, 95)
Train Rows: (23514, 42)
Test Rows: (2070, 42)


<div class="alert alert-block alert-info">
<font color=black><br>

- We can then see that we have **42** columns in each of the train and test sets, one for the chunk id, position within chunk, hour of day, and the **39** training variables. 
- Please note that we still I have not taken care of the **NaN** entries.

<br></font>
</div>

In [73]:
pd.DataFrame(test_rows)

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,1,121,21,7.109461,3.409452,NaN,0.114975,0.344926,0.114975,0.114975,...,4.205421,NaN,NaN,7.217472,7.91353,1.652703,2.722099,NaN,4.163092,NaN
1,1,122,22,7.109461,2.995219,NaN,0.114975,0.402413,0.114975,0.114975,...,3.37722,NaN,NaN,7.211707,8.875986,1.458267,2.430445,NaN,3.751923,NaN
2,1,123,23,7.067268,3.47318,NaN,0.114975,0.114975,0.114975,0.114975,...,3.653287,NaN,NaN,7.223236,9.39124,1.555485,2.138792,NaN,4.0603,NaN
3,1,124,0,5.252984,4.046732,NaN,0.114975,0.114975,0.114975,0.114975,...,4.012174,NaN,NaN,7.684417,8.137131,1.652703,1.847138,NaN,4.394375,NaN
4,1,125,1,5.252984,4.684013,NaN,0.114975,0.114975,0.114975,0.114975,...,4.122601,NaN,NaN,7.678652,8.730159,1.749921,1.555485,NaN,4.856941,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2065,210,130,18,NaN,0.0,NaN,1.839603,2.529454,NaN,2.356991,...,0.800594,NaN,0.266612,NaN,NaN,0.291653,0.291653,NaN,0.256981,NaN
2066,210,137,1,NaN,0.0,NaN,1.494677,1.954578,NaN,1.839603,...,0.156438,NaN,0.86649,NaN,NaN,0.291653,0.291653,NaN,0.282679,NaN
2067,210,144,8,NaN,0.063728,NaN,0.632363,1.207239,NaN,0.977289,...,0.524527,NaN,0.999796,NaN,NaN,0.291653,0.291653,NaN,0.693849,NaN
2068,210,168,8,NaN,0.095592,NaN,2.127041,2.701916,NaN,2.414479,...,0.257663,NaN,0.266612,NaN,NaN,0.291653,0.291653,NaN,0.565358,NaN


# Forecast evaluation

<div class="alert alert-block alert-info">
<font color=black><br>

- It is helpful to have a simpler format when evaluating forecasts. 
- For example, we will use the three-dimensional structure of `[chunks, variables, time]`, where variable is the target variable number from 0 to 38 and time is the lead time index from 0 to 9

<br></font>
</div>

In [74]:
def prepare_test_forecasts(test_chunks):
    """Prepare test forecast
    
    Convert the test dataset in chunks to [chunk][variable][time] format  
    """
    predictions = list()
    # enumerate chunks to forecast
    for rows in test_chunks:
        # enumerate targets for chunk
        chunk_predictions = list()
        for j in range(3, rows.shape[1]):
            yhat = rows[:, j]
            chunk_predictions.append(yhat)
        chunk_predictions = array(chunk_predictions)
        predictions.append(chunk_predictions)
    return array(predictions)

<div class="alert alert-block alert-info">
<font color=black><br>

- We will evaluate a model using the mean absolute error, or **MAE**. Where is the mean below?
- This is the metric that was used in the competition and is a **sensible choice given the non-Gaussian distribution of the target variables**.


- If a lead time contains no data in the test set (e.g. NaN), then no error will be calculated for that forecast. 
- If the lead time does have data in the test set but no data in the forecast, then the full magnitude of the observation will be taken as error. 
- Finally, if the test set has an observation and a forecast was made, then the absolute difference will be recorded as the error. 

<br></font>
</div>

In [76]:
def calculate_error(actual, predicted):
    # give the full actual value if predicted is nan
    if isnan(predicted):
        return abs(actual)
    # calculate abs difference
    return abs(actual - predicted)

<div class="alert alert-block alert-info">
<font color=black><br>

- The overall MAE will be calculated, but we will also calculate a MAE for each forecast lead time. This can help with model selection generally as some models may perform differently at different lead times.

- The evaluate_forecasts() function below implements this, calculating the MAE and per-lead time MAE for the provided predictions and expected values in `[chunk, variable, time]` format.

<br></font>
</div>

In [77]:
# evaluate a forecast in the format [chunk][variable][time]
def evaluate_forecasts(predictions, testset):
    lead_times = get_lead_times()
    total_mae, times_mae = 0.0, [0.0 for _ in range(len(lead_times))]
    total_c, times_c = 0, [0 for _ in range(len(lead_times))]
    # enumerate test chunks
    for i in range(len(test_chunks)):
        # convert to forecasts
        actual = testset[i]
        predicted = predictions[i]
        # enumerate target variables
        for j in range(predicted.shape[0]):
            # enumerate lead times
            for k in range(len(lead_times)):
                # skip if actual in nan
                if isnan(actual[j, k]):
                    continue
                # calculate error
                error = calculate_error(actual[j, k], predicted[j, k])
                # update statistics
                total_mae += error
                times_mae[k] += error
                total_c += 1
                times_c[k] += 1
    # normalize summed absolute errors
    total_mae /= total_c
    times_mae = [times_mae[i]/times_c[i] for i in range(len(times_mae))]
    return total_mae, times_mae

In [79]:
# summarize scores
def summarize_error(name, total_mae, times_mae):
    # print summary
    lead_times = get_lead_times()
    formatted = ['+%d %.3f' % (lead_times[i], times_mae[i]) for i in range(len(lead_times))]
    s_scores = ', '.join(formatted)
    print('%s: [%.3f MAE] %s' % (name, total_mae, s_scores))
    # plot summary
    pyplot.plot([str(x) for x in lead_times], times_mae, marker='.')
    pyplot.show()

# Data cleaning

<div class="alert alert-block alert-info">
<font color=black><br>

- Three candidate strategies for dealing with these gaps are as follows:
    - Ignore the gaps.
    - Use data without gaps.
    - Fill the gaps.

- Here we'll fill the gap using the median. This is a good choice given the non-normal distribution of the data.

<br></font>
</div>

In [81]:
# layout a variable with breaks in the data for missing positions
def variable_to_series(chunk_train, col_ix, n_steps=5*24):
    # lay out whole series
    data = [nan for _ in range(n_steps)]
    # mark all available data
    for i in range(len(chunk_train)):
        # get position in chunk
        position = int(chunk_train[i, 1] - 1)
        # store data
        data[position] = chunk_train[i, col_ix]
    return data

In [82]:
# interpolate series of hours (in place) in 24 hour time
def interpolate_hours(hours):
    # find the first hour
    ix = -1
    for i in range(len(hours)):
        if not isnan(hours[i]):
            ix = i
            break
    # fill-forward
    hour = hours[ix]
    for i in range(ix+1, len(hours)):
        # increment hour
        hour += 1
        # check for a fill
        if isnan(hours[i]):
            hours[i] = hour % 24
    # fill-backward
    hour = hours[ix]
    for i in range(ix-1, -1, -1):
        # decrement hour
        hour -= 1
        # check for a fill
        if isnan(hours[i]):
            hours[i] = hour % 24

# Input-output preparations

<div class="alert alert-block alert-info">
<font color=black><br>

-  

<br></font>
</div>

In [85]:
# test supervised to input/output patterns
from numpy import array
 
# created input/output patterns from a sequence
def supervised_for_lead_time(series, n_lag, lead_time):
    data = list()
    # enumerate observations and create input/output patterns
    for i in range(n_lag, len(series)):
        end_ix = i + (lead_time - 1)
        # check if can create a pattern
        if end_ix >= len(series):
            break
        # retrieve input and output
        start_ix = i - n_lag
        row = series[start_ix:i] + [series[end_ix]]
        data.append(row)
    return array(data)
 
# define test dataset
data = [x for x in range(20)]
# convert to supervised format
result = supervised_for_lead_time(data, 2, 6)
# display result
print(result)

[[ 0  1  7]
 [ 1  2  8]
 [ 2  3  9]
 [ 3  4 10]
 [ 4  5 11]
 [ 5  6 12]
 [ 6  7 13]
 [ 7  8 14]
 [ 8  9 15]
 [ 9 10 16]
 [10 11 17]
 [11 12 18]
 [12 13 19]]


In [87]:
# create supervised learning data for each lead time for this target
def target_to_supervised(chunks, rows, hours, col_ix, n_lag):
    train_lead_times = list()
    # get series
    series = variable_to_series(rows, col_ix)
    if not has_data(series):
        return None, [nan for _ in range(n_lag)]
    # impute
    imputed = impute_missing(chunks, rows, hours, series, col_ix)
    # prepare test sample for chunk-variable
    test_sample = array(imputed[-n_lag:])
    # enumerate lead times
    lead_times = get_lead_times()
    for lead_time in lead_times:
        # make input/output data from series
        train_samples = supervised_for_lead_time(imputed, n_lag, lead_time)
        train_lead_times.append(train_samples)
    return train_lead_times, test_sample

In [89]:
# prepare training [var][lead time][sample] and test [chunk][var][sample]
def data_prep(chunks, n_lag, n_vars=39):
    lead_times = get_lead_times()
    train_data = [[list() for _ in range(len(lead_times))] for _ in range(n_vars)]
    test_data = [[list() for _ in range(n_vars)] for _ in range(len(chunks))]
    # enumerate targets for chunk
    for var in range(n_vars):
        # convert target number into column number
        col_ix = 3 + var
        # enumerate chunks to forecast
        for c_id in range(len(chunks)):
            rows = chunks[c_id]
            # prepare sequence of hours for the chunk
            hours = variable_to_series(rows, 2)
            # interpolate hours
            interpolate_hours(hours)
            # check for no data
            if not has_data(rows[:, col_ix]):
                continue
            # convert series into training data for each lead time
            train, test_sample = target_to_supervised(chunks, rows, hours, col_ix, n_lag)
            # store test sample for this var-chunk
            test_data[c_id][var] = test_sample
            if train is not None:
                # store samples per lead time
                for lead_time in range(len(lead_times)):
                    # add all rows to the existing list of rows
                    train_data[var][lead_time].extend(train[lead_time])
        # convert all rows for each var-lead time to a numpy array
        for lead_time in range(len(lead_times)):
            train_data[var][lead_time] = array(train_data[var][lead_time])
    return array(train_data), array(test_data)

In [ ]:
#train_rows = array([row for rows in train for row in rows])
#test_rows = to_forecasts(test)

In [90]:
# group data by chunks
train_chunks = to_chunks(train_rows)
test_chunks = to_chunks(test_rows)

In [92]:
# convert training data into supervised learning data
n_lag = 12
train_data, test_data = data_prep(train_chunks, n_lag)
print(train_data.shape, test_data.shape)
## save train and test sets to file
#save('AirQualityPrediction/supervised_train.npy', train_data)
#save('AirQualityPrediction/supervised_test.npy', test_data)

KeyError: 0

# References

<div class="alert alert-block alert-warning">
<font color=black><br>

- https://machinelearningmastery.com/how-to-develop-machine-learning-models-for-multivariate-multi-step-air-pollution-time-series-forecasting/
- [Download dataset](https://www.kaggle.com/c/dsg-hackathon/data)

<br></font>
</div>